<img align="center" src="https://raw.githubusercontent.com/FUlyankin/Parsers/master/images%20/cats.jpg" height="1200" width="1200"> 

# <center> ML для маркетинга. <br>  <br> Cюжетная арка 3: сегментация </center>

В этом семестре мы с вами пройдём через несколько сюжетных арок. Третьей и заключительной такой аркой будет сегментация. 


#  Глава II (Вконтакте и его мальчик) 

Ознакомиться со способами взаимодействия с Вконтакте можно по [ссылке](https://vk.com/dev/manuals) на отличную документацию. Для того, чтобы получить доступ к выгрузке данных, нужно пройти ряд бюрократических процедур (о, боже, эти два предложения были так бюрократически сформулированы, что мне захотелось отстоять в очереди).

Первая такая процедура заключается в создании своего приложения. Для этого переходим по [ссылке](http://vk.com/editapp?act=create) и проходимся по необходимым шагам:

<img align="center" src="pictures/app_creation_1.png" height="600" width="600">

После подтверждения своей личности по номеру телефона, попадаем на страницу свежесозданного приложения
<img align="center" src="pictures/app_creation_2.png" height="600" width="600">

Слева нам будем доступна вкладка с настройками, перейдя в неё мы увидим все необходимые нам для работы с приложением параметры:
<img align="center" src="pictures/app_creation_3.png" height="600" width="600">


Для работы с частью методов API этого вполне достаточно (обычно в заголовке такого метода стоит соответствующая пометка). Для части методов, используемых нами, может понадобиться ключ доступа.

Для того, чтобы получить его, необходимо сделать ещё пару странных манипуляций:

Переходим по ссылке вида (на месте звездочек должен стоять ID созданного вами приложения):

> https://oauth.vk.com/authorize?client_id=**********&scope=14&redirect_uri=https://oauth.vk.com/blank.html&display=page&v=5.16&response_type=token

<img align="center" src="pictures/app_creation_4.png" height="600" width="600">

В итоге по этому запросу будет сформирована ссылка следующего вида:
> https://oauth.vk.com/blank.html#access_token=25b636116ef40e0718fe4d9f382544fc28&expires_in=86400&user_id=*******

Первый набор знаков - `access token`, т.е. маркер доступа. Вторая цифра (`expires_in=`) время работы маркера доступа в секундах (одни сутки). По истечению суток нужно будет получить новый маркер доступа. Последняя цифра (`user_id=`) ваш ID Вконтакте. Нам в дальнейшем понадобится маркер доступа. Для удобства сохраним его в отдельном файле или экспортируем в глобальную область видимости. В целях безопасности ваших данных не стоит нигде светить токенами и тем более выкладывать их в открытый доступ.

Обратите внимание на ссылку, по которой мы делали запрос на предоставление токена. Внутри неё находится странный параметр `scope=14.` Эта загадочная цифра есть ничто иное, как права доступа к социальной сети. Подробнее познакомиться с взаимно-однозначным соответствием между числами и правами можно в документации. Например, если мы хотим получить доступ к друзьям, фото и аудио, мы подставим в scope цифру 2+4+8=14.

> __ВАЖНО:__ Если отдать свой токен другому человеку, у него будет доступ к вашему аккаунту. Он легко может выкачать ваши личные переписки и даже удалить страницу. Не светите свой токен в интернетах и берегите его с молоду. 

# 1. Впервые врываемся в вк и узнаём как нас зовут

In [ ]:
import requests # библиотека для походов в интернет
import time     # пакет для работы с временем

Подгрузим токен.

In [ ]:
# мой номер странички
myid = '6045249'# вставить номер странички

# версия используемого API
version = '5.103' 

# подгружаем токен
with open('secret_token.txt') as f:
    token = f.read()

Чтобы скачать что-то из контакта, надо сделать ссылку и сходить по ней пакетом `requests`. Ссылка должна будет включать в себя метод (что мы просим у вк) и параметры (насколько много). Мы будем просто заменять эти две штуки и выкачивать разные вещи. 

In [ ]:
def vk_download(method, parameters):
    url = 'https://api.vk.com/method/' + method + '?' + parameters + '&access_token=' + str(token) + '&v=' + version
    response = requests.get(url) 
    infa = response.json()
    return infa 

In [ ]:
method = 'users.get'
parameters = 'user_ids=' + myid

vk_download(method, parameters)

Если вы всё сделали правильно, то у вас выводится имя и фамилия.

# 2.  Скачаем всех подписчиков пабликов с мемами

Мы хотели в следущей тетрадке поанализировать аудиторию разных пабликов с мемами и понять чем люди живут. Я выбрал для нас следушие паблики: 

**нормальные мемы**

* [Хаер скул оф мемес](https://vk.com/hsemem)
* [Мемы про машиное обучение для взрослых мужиков](https://vk.com/weirdreparametrizationtrick)
* [Физкек](https://vk.com/phys_kek)
* [Абстрактные мемы для элиты всех сортов](https://vk.com/abstract_memes)

**сомнительные мемы**

* [Я смотрю украинские шоу и радуюсь жизни](https://vk.com/ukrainskieshow)
* [Female memes](https://vk.com/femalemem)


**очень очень плохие мемы**

* [Чёткие приколы](https://vk.com/oroom)
* [Остроумные](https://vk.com/ostroym)
* [Кривое зеркало (театр Евгения Петросяна)](https://vk.com/teleteatrkrivoezerkalo)



In [ ]:
publics = {
    'hse_memes': '139105204',
    'ml_memes': 'weirdreparametrizationtrick',
    'fizkek': '111557607',
    'amdevs' : '92337511',
    
    'female_memes': '139923997',
    'ukr_memes': '111869067',
    
    'bad_memes1': '41807730',
    'bad_memes2': '31836774',
    'wtf_memes': 'teleteatrkrivoezerkalo'    
}

С каждой группы скачаем $50000$. По умлочанию все участники группы отсоритрованы по своему номеру id. Чем больше id, тем позднее зерегестрировался человек (поначалу id выдавались по порядку). Для того, чтобы срезать из группы свежих юзеров, давайте выкачивать их с большим значением `offset`. 

Начнём с вышки.

In [ ]:
cur_id = publics['hse_memes']

In [ ]:
# узнаем сколько в группе человек
cur_infa = vk_download('groups.getMembers', 'group_id=' + str(cur_id))
cur_infa['response']['count']

По числу людей в группе будем выбирать откуда начинать скачку. Если в группе $111 000$, тогда `offset = 111 000 - 50000.` Чтобы не уходить в отрицательные числа, если группа маленькая, будем брать максимум из $0$ и нашего отсупа.

In [ ]:
cur_infa['response']['count'] - 50000

In [ ]:
start_offset = max(0, cur_infa['response']['count'] - 50000)
start_offset

Скачиваем! 

In [ ]:
%%time
infa = [ ] # сюда буду сохранять людей

for offset in range(start_offset, start_offset + 50000, 1000):
    
    time.sleep(0.4) # вк согласен работать 3 раза в секунду, 
                    # эта строка нужна, чтобы он не ругался 
    
    cur_infa = vk_download('groups.getMembers', 
                'group_id=' + str(cur_id) + '&offset=' + str(offset))

    infa.extend(cur_infa['response']['items'])

In [ ]:
len(infa)

Завернём наш цикл с пары в функцию. И применим её по очереди к каждому паблику. 

In [ ]:
def group_members_download(cur_id):
    
    # Сколько всего в группе людей
    cur_infa = vk_download('groups.getMembers', 'group_id=' + str(cur_id))
    
    # стартовая точка
    start_offset = max(0, cur_infa['response']['count'] - 50000)

    infa = [ ] # сюда буду сохранять людей

    # качаем по 1000 штук по циклу :) 
    for offset in range(start_offset, start_offset + 50000, 1000):

        time.sleep(0.4) # вк согласен работать 3 раза в секунду, 
                        # эта строка нужна, чтобы он не ругался 

        cur_infa = vk_download('groups.getMembers', 
                    'group_id=' + str(cur_id) + '&offset=' + str(offset))

        infa.extend(cur_infa['response']['items'])
    return infa

Давайте скачивать. 

In [ ]:
%%time
members = { } # сюда будем класть всех членов групп по очереди

for k,v in publics.items():
    print('Качаю группу', k)
    infa = group_members_download(v)
    members[k] = infa    

На всякий случай сохраним всех, кого скачали. Будет обидно потерять их и скачивать заново. 

In [ ]:
import pickle  # пакет для сохранения и подгрузки питонячих объектов

# сохраняем всё в файл group_members.pickle
with open('group_members.pickle', 'wb') as f:
    pickle.dump(members, f)
    
# Подгрузить данные из файлика можно вот так: 
# with open('group_members.pickle', 'rb') as f:
#     members = pickle.load(f)

# 3. Отфильтровываем плохих подписчиков

Давайте выбросим из нашей выборки всех людей, которые: 

- Не заходили вконтакт в течение последлних 30 дней
- Удалили свою страницу 
- Закрыли свой профиль от внещних глаз

Узнать это всё можно, если найти соотвествующие [fields в документации.](https://vk.com/dev/objects/user) Про удаление страницы и закрытый профиль информация возвращается по умолчанию во вкладках `deactivated` и `is_closed`. Инфу про заходы надо тащить дополнительно с полем `last_seen`.

In [ ]:
infa = vk_download('users.get','user_ids=6045249,494821147&fields=last_seen')
infa

Время последнего захода указывается в виде количества секунд, прошедшего с 1 января 1970 года. Это везде так, привыкайте. 

In [ ]:
# Пакет time позволяет посмотреть сколько секунд прошло к текущему моменту. 
time.time()

Сколько секунд в $30$ днях? 

In [ ]:
30*24*60*60

In [ ]:
# Это дата 30-дневной давности :) 
time.time() - 30*24*60*60

Пробуем скачать информацию! Документация говорит, что максимально в `user_ids` можно пихать $1000$ id. Снова придётся писать цикл. Напишем его для какой-нибудь группы, а потом завернём в функцию. 

In [ ]:
memes_id = members['hse_memes']  # положили все id в вектор 
len(memes_id)

Всего у нас 50 тыщ людей. Запихнём первую тыщу в функцию для скачивания. Перечислить все id в виде текста через запятую можно вот так: 

In [ ]:
# сделали каждый id текстом
memes_id_text = [str(item) for item in memes_id]
memes_id[0] # цифра  

In [ ]:
memes_id_text[0] # текст

In [ ]:
# перечислили их через запятую
','.join(memes_id_text[:10])

Скачаем инфу по шестнадцатой трёхсотне :3 

In [ ]:
infa = vk_download('users.get','user_ids=' + ','.join(memes_id_text[16*300:17*300]) + '&fields=last_seen')

In [ ]:
infa['response'][0]

Просеиваем плохие id. 

In [ ]:
clean_member_id = [ ] # сюда будем записывать хорошие

for item in infa['response']:
    
    # если пользователь не удален, не приватный, а ещё недавно был в сети 
    if ('deactivated' not in item) and \
       (item['is_closed'] == False) and \
       (item['last_seen']['time'] >= time.time() - 30*24*60*60):
    
                clean_member_id.append(item) # запоминаем его 

АААААА НЕ РАБОТАЕТ!!! Посмотрим на ком именно произошла ошибка. 

In [ ]:
item

У пользователя не скачалась информация о том как давно он заходил в сеть. Видимо, это происходило очень давно. Давайте будем выбрасывать таких людей из выборки. Нам в этом поможет конструкция `try-except`.

In [ ]:
clean_member_id = [ ] # сюда будем записывать хорошие

for item in infa['response']:
    
    try: # попробуй сделать вот это
        
        # если пользователь не удален, не приватный, а ещё недавно был в сети 
        if ('deactivated' not in item) and \
           (item['is_closed'] == False) and \
           (item['last_seen']['time'] >= time.time() - 30*24*60*60):

                    clean_member_id.append(item) # запоминаем его 
    
    except: # если питон говорит, что ошибся, сделай вот это 
        print(item)

Одного человека мы просто игнорируем. Не большая потеря для выборки. 

In [ ]:
len(clean_member_id) # из 300 осталось 294

Это надо сделать со всеми id из списка, который мы скачали. Напишем для этого функцию и цикл. 

In [ ]:
def filter_members(memes_id):
    
    # сделали id текстом
    memes_id_text = [str(item) for item in memes_id]
    
    clean_memes_id = [ ] # список для хорших id 
    
    # качаем информацию кусочками по 300 штук
    for i in range(len(memes_id_text)//300):
                
        time.sleep(0.4) # немного подождали, чтобы вк нас не банил
        
                                                         # взяли текущие 300 человек из списка
        infa = vk_download('users.get','user_ids=' + ','.join(memes_id_text[i*300:(i+1)*300]) + '&fields=last_seen')
 
        

        # чистим пользователей
        for item in infa['response']:

            try: # попробуй сделать вот это

                # если пользователь не удален, не приватный, а ещё недавно был в сети 
                if ('deactivated' not in item) and \
                   (item['is_closed'] == False) and \
                   (item['last_seen']['time'] >= time.time() - 30*24*60*60):

                            clean_memes_id.append(item) # запоминаем его 

            except:   # если питон говорит, что ошибся, сделай вот это 
                pass  # (ничего не делай) 
    

    # можно выводить инфу на экран вот так
    print('остаось {} человек'.format(len(clean_memes_id)))
    
    return clean_memes_id # вернули из функции чистые id

Применяем эту функцию ко всем группам. 

In [ ]:
%%time
clean_members = { } # сюда будем класть всех прочищеных членов групп по очереди

for k,v in members.items():
    print('Фильтрую id из группы', k)
    infa = filter_members(v)
    clean_members[k] = infa    

__Первый вывод:__ манишлернеры какие-то хикки, закрывают свои страницы. А в группу кривого зеркала никто похоже не заходит. Петросян нагнал ботов. Снова на всякий случай сохраним все id. 

In [ ]:
with open('group_clean_members.pickle', 'wb') as f:
    pickle.dump(clean_members, f)

# 4. Скачиваем стены

Для каждого пользователя скачаем со стены $100$ записей. Снова если вк не разрешает нам сделать этого, будем игнорировать id. Приватные аккаунты всё ещё остались в выборке :( 

In [ ]:
import pickle
with open('group_clean_members.pickle', 'rb') as f:
    clean_members = pickle.load(f)

In [ ]:
cur_id = clean_members['hse_memes'][10]['id']
cur_id

In [ ]:
wall = vk_download('wall.get', 'owner_id='+str(cur_id)+'&count=100')
wall = wall['response']['items']
len(wall)

In [ ]:
wall[0]

Будем как на паре вытаскивать тексты постов и тексты репостов. 

In [ ]:
post_texts = [cur['text'] for cur in wall]
repost_texts = [cur['copy_history'][0]['text'] if 'copy_history' in cur else '' for cur in wall]

Их будем соединять в один огромный пост. 

In [ ]:
wall_text = ' '.join(post_texts) + ' '.join(repost_texts)
wall_text

Все такие стены будем сохранять. Для этого напишем функцию и цикл. 

In [ ]:
def get_wall(user_id):
    
    # скачели стену 
    wall = vk_download('wall.get', 'owner_id='+str(user_id)+'&count=100')
    
    # распаковали её
    wall = wall['response']['items']
    post_texts = [cur['text'] for cur in wall]
    repost_texts = [cur['copy_history'][0]['text'] if 'copy_history' in cur else '' for cur in wall]
    
    # соединили все посты и репосты в один длинный текст
    wall_text = ' '.join(post_texts) + ' '.join(repost_texts)
    return wall_text

Из каждого паблика скачаем $1000$ рандомных стенок.  

__ВАЖНО:__ на метод `wall.get` есть ограничение! Можно скачивать не более $5000$ стен в сутки. Когда вы пробьёте его, вы увидете ошибку 

```
{'error': {'error_code': 29,
  'error_msg': 'Rate limit reached',
  'request_params': [{'key': 'owner_id', 'value': '43152073'},
   {'key': 'count', 'value': '100'},
   {'key': 'v', 'value': '5.103'},
   {'key': 'method', 'value': 'wall.get'},
   {'key': 'oauth', 'value': '1'}]}}
```

Нужно будет сделать в скачке перерыв. __Лучше качайте паблики по одному и сохраняйте в файлики по одному! Цикл для этого переписать вы в состоянии :)__

In [ ]:
import numpy as np

for k,v in list(clean_members.items())[1:]:
    print('Качаю стены участников из группы', k)    
    wall_members = [ ] # сюда будем сохранять стенки
    
    cur_members = np.array(v) 
    cur_sample = np.random.choice(cur_members, 1000, replace = False)
    
    for item in cur_sample:
    
        time.sleep(0.4) # отдыхаем между запросами 
        
        # попробуй, если код упал, напечатай мне id пользователя
        try:
            # распаковываем стену пользователя 
            wall_text = get_wall(item['id'])

            cur_item = {'group': k, 'id': item['id'], 'wall': wall_text}

            wall_members.append(cur_item)
        except:
            print('Ошибка!', item) 

    # сохраняем итоговые тексты стенок для текущего сообщества 
    with open(k + '.pickle', 'wb') as f:
        pickle.dump(wall_members, f)


In [ ]:
!ls -hl

Этот код будет работать довольно долго. Скачка будет идти со скоростью примерно $10000$ записей со стенок в час. Но тем не менее, он скачает нам всё, что нужно (вам в вашей работе $10000$ стенок более, чем достаточно для анализа.

In [ ]:
# wall_members[-1]

In [ ]:
# сохраняем итоговые тексты стенок
with open('group_clean_members.pickle', 'wb') as f:
    pickle.dump(wall_members, f)

## 5. Полезные хитрости


#### Хитрость 1:  Не стесняйтесь пользоваться `try-except`

Эта конструкция позволяет питону в случае ошибки сделать что-нибудь другое либо проигнорировать её. Например, мы хотим найти логарифм от всех чисел из списка: 

In [ ]:
from math import log 

a = [1,2,3,-1,-5,10,3]

for item in a:
    print(log(item))

У нас не выходит, так как логарифм от отрицательных чисел не берётся. Чтобы код не падал при возникновении ошибки, мы можем его немного изменить: 

In [ ]:
from math import log 

a = [1,2,3,-1,-5,10,3]

for item in a:
    try:
        print(log(item))  # попробуй взять логарифм
    except:
        print('я не смог') # если не вышло, сознайся и работай дальше

Чудо! Код больше не падает.

Как это использовать в вашем парсере контакта? Ну, например, если вы очень сильно боитесь, что код ночью может упасть из-за какого-то одного стремного пользователя вк, его скачку внутри цикла можно переписать вот так: 

Тогда в случае, если id будет битым, код не упадет, а продолжит работу, просто напечатав на экране, что в конкретном id произошла ошибка. Все остальные данные скачаются. Если вы боитесь, что компьютер может зависнуть ночью и умереть на половине скачки, добавьте внутрь цикла код, который будет сохранять данные по кусочкам, а не все сразу в конце скачки. 

#### Хитрость 2:  используйте пакет tqdm

> Код уже работает час. Я вообще без понятия когда он закончит работу. Было бы круто узнать, сколько ещё ждать... 

Если в вашей голове возникла такая мысль, пакет `tqdm` ваш лучший друг. Установить его можно так: 

In [ ]:
!pip install tqdm

Просто один раз запустите эту ячейку и `pip` (менеджер пакетов для питона) поставит его вам на компьютер. Теперь можно пользоваться вот такой штукой: 

In [ ]:
from tqdm import tqdm_notebook

a = list(range(30))

# 30 раз будем спать по секунде
for i in tqdm_notebook(a):
    time.sleep(1)

Мы обмотали тот вектор, по которому идёт цикл в `tqdm_notebook`. Это даёт нам красивую зелёную строку, которая показывает насколько сильно мы продвинулись по коду. Обматывайте свои самые большие и долгие циклы в `tqdm_notebook` и всегда понимайте сколько осталось до конца. 

#### Хитрость 3:  аккуратная работа с массивами

Если объявить список, куда вы закидываете людей в одной ячейке, а цикл запускать в другой, можно упростить себе жизнь. Смотрите как: 

In [ ]:
a = list(range(100)) # это 100 id которые я хочу скачать 

In [ ]:
b = [ ] # сюда я типо качаю по ним инфу

In [ ]:
for i in a:
    b.append(i**2)  # это я типо скачал её и добавил в b
    
    # это я сымитировал падение на 50 id (цикл просто прекратит работать)
    if i == 50:
        break

In [ ]:
len(b) # скачали 51 человека, нужны еще 49

В `b` уже записана информации по $51$ человеку. Давайте просто продолжим скачку тем же циклом, но с 52 человека!

In [ ]:
for i in a[51:]:
    b.append(i**2)  # это я типо скачал её и добавил в b
    
    # это я сымитировал падение на 50 id (цикл просто прекратит работать)
    if i == 50:
        break

In [ ]:
len(b)  # всё скачалось! 

Надеюсь вы поняли о чём я. Когда код упал в середине списка для скачки, не обязательно запускать его с самого начала! Половина то уже скачалась и её можно сохранить. 

#### Хитрость 4:  ускорение кода в 25 раз

При желании ваш код можно ускорить в $25$ раз. Для этого вам надо изучить самим [туториал по тому как качать вконтакте,](https://nbviewer.jupyter.org/github/FUlyankin/ekanam_grand_research/blob/master/0.%20vk_parser_tutorial.ipynb) В нём описано как можно переделать код в более быстрый. Можно взять код для скачки [отсюда,](https://github.com/DmitrySerg/OpenData/blob/master/RussianElections2018/Part_1_Parsing_VK.ipynb) он уже переделан. Но придётся заставить его работать. 

__Обратите внимание, что это не поможет вам пробить лимиты, которые вк устанавливает на работу разных методов скачки.__